In [1]:
import pandas as pd
import numpy as np
from datetime import date

import pandas_datareader.data as web
import pandas_datareader as pdr

In [2]:
START_DATE = '1999-01-01'
END_DATE = '2021-10-31'
TICKS_FX = ['DEXUSUK','DEXUSEU','DEXSZUS','DEXJPUS']
TICKS_RF = ['USD3MTD156N','GBP3MTD156N','EUR3MTD156N','CHF3MTD156N','JPY3MTD156N']

rfraw = pdr.DataReader(TICKS_RF,data_source='fred', start=START_DATE,end=END_DATE)
fxraw = pdr.DataReader(TICKS_FX,data_source='fred', start=START_DATE,end=END_DATE)

In [3]:
info = pd.DataFrame(columns=['Ticker Description'])
info.loc['USD3M'] = '3-Month LIBOR, USD'
info.loc['GBP3M'] = '3-Month LIBOR, GBP'
info.loc['EUR3M'] = '3-Month LIBOR, EUR'
info.loc['CHF3M'] = '3-Month LIBOR, CHF'
info.loc['JPY3M'] = '3-Month LIBOR, JPY'

info.loc['USUK'] = 'Spot FX, US per UK (GBP)'
info.loc['USEU'] = 'Spot FX, US per EU (EUR)'
info.loc['USSZ'] = 'Spot FX, US per SZ (CHF)'
info.loc['USJPY'] = 'Spot FX, US per JP (JPY)'
info

,Ticker Description
USD3M,"3-Month LIBOR, USD"
GBP3M,"3-Month LIBOR, GBP"
EUR3M,"3-Month LIBOR, EUR"
CHF3M,"3-Month LIBOR, CHF"
JPY3M,"3-Month LIBOR, JPY"
USUK,"Spot FX, US per UK (GBP)"
USEU,"Spot FX, US per EU (EUR)"
USSZ,"Spot FX, US per SZ (CHF)"
USJPY,"Spot FX, US per JP (JPY)"


In [4]:
rf = pd.DataFrame(index=rfraw.index)
for tick in TICKS_RF:
    lab = tick[0:5]
    rf[lab] = rfraw[tick]/100

rf = rf.resample('M').last()
rf

,USD3M,GBP3M,EUR3M,CHF3M,JPY3M
DATE,,,,,
1999-01-31,0.049695,0.058159,0.030738,0.012717,0.004781
1999-02-28,0.050263,0.054616,0.030987,0.012617,0.002744
1999-03-31,0.050000,0.053241,0.029676,0.012483,0.001872
1999-04-30,0.049887,0.053312,0.025838,0.009933,0.001350
1999-05-31,0.050662,0.053731,0.025800,0.010233,0.000937
...,...,...,...,...,...
2021-06-30,0.001457,0.000779,-0.005511,-0.007536,-0.000775
2021-07-31,0.001177,0.000740,-0.005541,-0.007594,-0.001030
2021-08-31,0.001196,0.000684,-0.005594,-0.007704,-0.000987


In [5]:
fx = pd.DataFrame(index=fxraw.index)

for tick in TICKS_FX:
    lab = tick[-4:]
    if lab[-2:] == 'US':
        lab = lab[-2:] + lab[-4:-2]
        fx[lab] = 1/fxraw[tick]
    else:
        fx[lab] = fxraw[tick]
    
fx = fx.resample('M').last()
fx

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,1.6457,1.1371,0.705816,0.008621
1999-02-28,1.6027,1.0995,0.689893,0.008425
1999-03-31,1.6140,1.0808,0.676819,0.008444
1999-04-30,1.6085,1.0564,0.655437,0.008373
1999-05-31,1.6020,1.0422,0.654450,0.008273
...,...,...,...,...
2021-06-30,1.3806,1.1848,1.080731,0.009005
2021-07-31,1.3913,1.1864,1.103631,0.009116
2021-08-31,1.3747,1.1800,1.091584,0.009087


# Save to Excel

In [6]:
with pd.ExcelWriter('fx_carry_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name = 'descriptions')
    rf.to_excel(writer, sheet_name= 'risk-free rates')
    fx.to_excel(writer, sheet_name='fx rates')